# Experiment 01-02: Hashcat

Test if the Hashcat SHA256 kernel is loading.

In [9]:
import sys
from glob import glob
import numpy as np
import pyopencl

## Helper Functions

In [10]:
def get_first_platform_by_name(name : str) -> pyopencl.Platform:
    """
    Get the first platform with the given name.

    Parameters
    ----------
    name : str
        The name of the platform to get.
    """
    for platform in pyopencl.get_platforms():
        if platform.name == name:
            return platform

    raise ValueError(f"No platform named '{name}'")

In [11]:
def build_program(program_files : list, context : pyopencl.Context,
        build_options=[]) -> pyopencl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    context : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source_code = ''

    for ocl_file in program_files:
        with open(ocl_file, 'r') as opencl_file:
            file_source_code = opencl_file.read()
            program_source_code += '\n' + file_source_code

    program_source = pyopencl.Program(context, program_source_code)
    program = program_source.build(options=build_options)
            
    return program

In [12]:
def run_ocl_kernel(queue, kernel, global_size,
                   input_tuples, output_tuples,
                   local_size = (32,)):
    
    # copying data onto the device
    for (value, buffer) in input_tuples:
        pyopencl.enqueue_copy(queue, src=value, dest=buffer)
    
    # running program on the device
    kernel_arguments  = [buffer for (_,buffer) in input_tuples] 
    kernel_arguments += [buffer for (_,buffer) in output_tuples]
        
    kernel(queue, global_size, local_size, *kernel_arguments)

    # copying data off the device
    for (value, buffer) in output_tuples:
        pyopencl.enqueue_copy(queue, src=buffer, dest=value)
        
    # waiting for everything to finish
    queue.finish()

## Setup the device and context

In [13]:
ocl_platforms = (platform.name 
                 for platform in pyopencl.get_platforms())
print("\n".join(ocl_platforms))

Apple


In [14]:
gpu_platform = get_first_platform_by_name("Apple")
gpu_devices = gpu_platform.get_devices(device_type=pyopencl.device_type.GPU)
gpu_context = pyopencl.Context(devices=gpu_devices)

print(gpu_devices)
print(gpu_context)

[<pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>]
<pyopencl.Context at 0x600002b28420 on <pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>>


## Load Hashcat SHA256 kernel

In [15]:
# get a list of the source files
program_files = glob('../hashcat/OpenCL/*.cl')
print(program_files)

['../hashcat/OpenCL/inc_common.cl', '../hashcat/OpenCL/inc_platform.cl', '../hashcat/OpenCL/inc_hash_sha256.cl']


In [16]:
# build the program
build_options = [
    #'-I', '../hashcat/include/',
    '-I', '../hashcat/OpenCL/',
]

gpu_program = build_program(program_files, gpu_context, build_options)

print(gpu_program)

RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>:

Compilation failed: 

In file included from program_source:8:
/Users/foulds/code/dsm070-2022-apr/topic_06/hashing/experiment01-hello_opencl/../hashcat/OpenCL/inc_types.h:74:9: error: unknown type name 'uint8_t'; did you mean 'uint8'?
typedef uint8_t  u8;
        ^~~~~~~
        uint8
/System/Library/PrivateFrameworks/GPUCompiler.framework/Versions/31001/Libraries/lib/clang/31001.525/include/opencl-c.h:113:14: note: 'uint8' declared here
typedef uint uint8 __attribute__((ext_vector_type(8)));
             ^
In file included from program_source:8:
/Users/foulds/code/dsm070-2022-apr/topic_06/hashing/experiment01-hello_opencl/../hashcat/OpenCL/inc_types.h:75:9: error: unknown type name 'uint16_t'; did you mean 'uint16'?
typedef uint16_t u16;
        ^~~~~~~~
        uint16
/S